# Analyzing Online Ticket Sales with Amazon Redshift

In this workspace, we will be accessing data stored in Amazon Redshift, a data warehouse product that is part of Amazon Web Services. More specifically, we'll be analyzing sales activity from a fictional ticketing website where users both buy and sell tickets online for sporting events, shows, and concerts ([source](https://docs.aws.amazon.com/redshift/latest/dg/c_sampledb.html)).

## Explore events

In [19]:
-- List all the events
SELECT * FROM event

,eventid,venueid,catid,dateid,eventname,starttime
0,423,303,8,1827,La Damnation de Faust,2008-01-01 19:30:00+00:00
1,2099,211,7,1827,The Bacchae,2008-01-01 19:30:00+00:00
2,6640,38,9,1827,Herbie Hancock,2008-01-01 19:30:00+00:00
3,8621,9,9,1827,Hot Chip,2008-01-01 19:00:00+00:00
4,1376,235,6,1827,The King and I,2008-01-01 14:30:00+00:00
...,...,...,...,...,...,...
8793,6034,45,9,2191,War,2008-12-31 14:00:00+00:00
8794,6783,60,9,2191,The Police,2008-12-31 15:00:00+00:00
8795,6857,18,9,2191,Judas Priest,2008-12-31 14:00:00+00:00
8796,7192,54,9,2191,Lindsey Buckingham,2008-12-31 19:30:00+00:00


This is linking up to several other tables in the warehouse, such as venue, category and date. Let's join things up.

In [21]:
SELECT *
FROM event
INNER JOIN venue USING(venueid)
INNER JOIN category USING(catid)
INNER JOIN date USING(dateid)
LIMIT 100

,dateid,catid,venueid,eventid,eventname,starttime,venuename,venuecity,venuestate,venueseats,catgroup,catname,catdesc,caldate,day,week,month,qtr,year,holiday
0,1827,8,303,423,La Damnation de Faust,2008-01-01 19:30:00+00:00,Grand 1894 Opera House,Galveston,TX,0.0,Shows,Opera,All opera and light opera,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
1,1827,7,211,2099,The Bacchae,2008-01-01 19:30:00+00:00,The Broadway Theatre,New York City,NY,0.0,Shows,Plays,All non-musical theatre,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
2,1827,9,38,6640,Herbie Hancock,2008-01-01 19:30:00+00:00,US Airways Center,Phoenix,AZ,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
3,1827,9,9,8621,Hot Chip,2008-01-01 19:00:00+00:00,Dick's Sporting Goods Park,Commerce City,CO,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
4,1827,6,235,1376,The King and I,2008-01-01 14:30:00+00:00,Studio 54,New York City,NY,0.0,Shows,Musicals,Musical theatre,2008-01-01 00:00:00+00:00,WE,1,JAN,1,2008,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1830,9,9,8006,Bette Midler,2008-01-04 19:30:00+00:00,Dick's Sporting Goods Park,Commerce City,CO,0.0,Concerts,Pop,All rock and pop music concerts,2008-01-04 00:00:00+00:00,SA,2,JAN,1,2008,False
96,1830,6,244,982,Gypsy,2008-01-04 15:00:00+00:00,Royce Hall,Los Angeles,CA,0.0,Shows,Musicals,Musical theatre,2008-01-04 00:00:00+00:00,SA,2,JAN,1,2008,False
97,1830,6,257,1567,Blue Man Group,2008-01-04 15:00:00+00:00,Luxor Hotel,Las Vegas,NV,NaN,Shows,Musicals,Musical theatre,2008-01-04 00:00:00+00:00,SA,2,JAN,1,2008,False
98,1830,7,241,2606,The Caretaker,2008-01-04 14:30:00+00:00,Curran Theatre,San Francisco,CA,0.0,Shows,Plays,All non-musical theatre,2008-01-04 00:00:00+00:00,SA,2,JAN,1,2008,False


There's a `starttime` column coming from the `event` table and there's also a `caldate` column, coming from the `date` table. Let's see what's up with this.

In [20]:
SELECT
	CASE WHEN DATE(caldate) = DATE(starttime) THEN True ELSE False END AS same_date,
	COUNT(*)
FROM event
INNER JOIN date USING(dateid)
GROUP BY 1

,same_date,count
0,True,8095
1,False,703


In [22]:
SELECT MAX(DATEDIFF('hour', caldate, starttime))
FROM event
INNER JOIN date USING(dateid)

,max
0,20


Let's see how much events are happening in different cities.

In [23]:
SELECT 
	venuecity,
    COUNT(*) AS num_event
FROM event
INNER JOIN venue USING(venueid)
GROUP BY 1
ORDER BY 2 DESC

,venuecity,num_event
0,New York City,2647
1,Los Angeles,312
2,Las Vegas,300
3,Chicago,209
4,San Francisco,194
...,...,...
74,Newark,27
75,Montreal,27
76,Irving,25
77,Sunrise,24


In [24]:
import plotly.express as px
px.bar(events_per_city, x = 'venuecity', y = 'num_event')

## Explore listings and sales

In [25]:
-- show 100 listing records
SELECT * FROM public.listing
LIMIT 100

,listid,sellerid,eventid,dateid,numtickets,priceperticket,totalprice,listtime
0,1315,37302,920,1827,9,126.0,1134.0,2008-01-01 04:05:41+00:00
1,724,35016,3468,1827,10,40.0,400.0,2008-01-01 03:32:37+00:00
2,1825,45077,3181,1827,16,118.0,1888.0,2008-01-01 01:16:37+00:00
3,7266,45195,7721,1827,2,91.0,182.0,2008-01-01 07:55:03+00:00
4,4118,40141,5624,1827,16,43.0,688.0,2008-01-01 03:10:06+00:00
...,...,...,...,...,...,...,...,...
95,86005,11470,654,1827,8,56.0,448.0,2008-01-01 12:13:49+00:00
96,110942,43282,7003,1827,4,354.0,1416.0,2008-01-01 06:20:18+00:00
97,87225,8039,4077,1827,4,150.0,600.0,2008-01-01 09:45:27+00:00
98,111755,2545,6286,1827,3,382.0,1146.0,2008-01-01 11:30:57+00:00


In [26]:
-- show 100 sales records
SELECT * FROM public.sales
LIMIT 100

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,97197,110942,43282,515,7003,1827,2,708.0,106.20,2008-01-01 02:30:52+00:00
1,110328,126347,6955,394,6213,1827,1,347.0,52.05,2008-01-01 01:00:19+00:00
2,165890,222879,2186,664,6870,1827,2,4192.0,628.80,2008-01-01 12:59:34+00:00
3,65082,73790,20429,451,1150,1827,4,472.0,70.80,2008-01-01 06:06:57+00:00
4,7011,7613,5933,1503,4515,1828,1,177.0,26.55,2008-01-02 01:52:35+00:00
...,...,...,...,...,...,...,...,...,...,...
95,72185,82279,34162,3538,8582,1831,2,726.0,108.90,2008-01-05 10:31:47+00:00
96,141506,163669,42762,1940,6118,1831,1,95.0,14.25,2008-01-05 03:24:34+00:00
97,82322,93881,6896,1417,3293,1831,1,186.0,27.90,2008-01-05 11:23:21+00:00
98,145178,168070,47124,562,6515,1831,2,600.0,90.00,2008-01-05 01:01:40+00:00


Let's see if multiple sales can happen for the same listing.

In [27]:
WITH listings_with_sales AS (
    SELECT 
        listid,
        COUNT(*) AS number_of_sales
    FROM public.listing
	INNER JOIN public.sales USING(listid)
    GROUP BY 1
)
SELECT
	number_of_sales,
    COUNT(*) AS num_listings
FROM listings_with_sales
GROUP BY 1
ORDER BY 1

,number_of_sales,num_listings
0,1,48029
1,2,36570
2,3,14665
3,4,1808
4,5,12


It turns out the bulk of listings have only one sale associated with them. There are 12 listings that had 5 sales.

We can find the user that sold the most tickets in 2008:

In [28]:
SELECT
	sellerid,
    username,
    (firstname ||' '|| lastname) as name,
	city,
    sum(qtysold)
FROM sales
INNER JOIN date USING(dateid)
INNER JOIN users ON sales.sellerid = users.userid
WHERE year = 2008
GROUP BY 1, 2, 3, 4
ORDER BY 5 desc
LIMIT 5

,sellerid,username,name,city,sum
0,48950,TUT90BHI,Nayda Hood,Frisco,46
1,19123,DZW00VOQ,Scott Simmons,Carson,41
2,20029,RPM45HGY,Drew Mcguire,Lancaster,41
3,36791,DCE77DOA,Emerson Delacruz,Springfield,40
4,9697,GDM25KSM,Dorian Ray,Vicksburg,39


Similarly, we can find the most active buyer on the site in 2008:

In [29]:
select
	buyerid,
    username,
    (firstname ||' '|| lastname) as name,
	city,
    sum(qtysold)
FROM sales
INNER JOIN date USING(dateid)
INNER JOIN users ON sales.buyerid = users.userid
WHERE year = 2008
GROUP BY 1, 2, 3, 4
ORDER BY 5 desc
LIMIT 5

,buyerid,username,name,city,sum
0,8933,CNF70VPH,Jerry Nichols,Middlebury,67
1,1298,EDB46JXK,Kameko Bowman,Newburyport,64
2,3797,KTV94TWB,Armando Lopez,Pomona,64
3,5002,CBC51API,Kellie Savage,Falls Church,63
4,3881,XJN46RCL,Herrod Sparks,Rome,60


Let's see if there's a big difference in average sales price for different categories of events. We're looking at actual sales here, not listings!

In [31]:
SELECT
	catgroup,
    AVG(pricepaid / qtysold) AS avg_ticket_price,
	MEDIAN(pricepaid / qtysold) AS median_ticket_price
FROM sales
INNER JOIN event USING(eventid)
INNER JOIN category USING(catid)
GROUP BY 1

,catgroup,avg_ticket_price,median_ticket_price
0,Concerts,333.755006,229.0
1,Shows,336.982704,232.0


Are there listings where the sale happened before the listing?

In [32]:
SELECT COUNT(*)
FROM listing
INNER JOIN sales USING(listid)
WHERE listtime > saletime

,count
0,2965


This must be bad data! Let's keep these out when figuring out the shortest and longest time to get a listing sold.

In [34]:
WITH tts AS (
    SELECT DATEDIFF('seconds', listtime, saletime) AS time_to_sell
    FROM sales
    INNER JOIN listing USING(listid)
    WHERE listtime < saletime
)
SELECT 
	MIN(time_to_sell) AS shortest_time_to_sell_in_seconds,
    MAX(time_to_sell)/3600/24 AS longest_time_to_sell_in_days
FROM tts

,shortest_time_to_sell_in_seconds,longest_time_to_sell_in_days
0,17,60


## Finding users that should advertise

Suppose we, as owners of the ticketing website, want to target certain users with the suggestion to advertise their listings, so they have a higher chance of actually getting sales. Let's build up a list of users that had the most outstanding listings in terms of price per ticket.

In [35]:
WITH listings_with_sales AS (
    SELECT 
        listid,
 	    listing.sellerid,
        numtickets AS tickets_listed,
        priceperticket,
		SUM(COALESCE(qtysold, 0)) AS tickets_sold
    FROM listing
	LEFT JOIN sales USING(listid)
    GROUP BY 1, 2, 3, 4
)
SELECT 
	sellerid,
    (firstname ||' '|| lastname) as name,
	SUM((tickets_listed - tickets_sold) * priceperticket)  AS unrealized_sales
FROM listings_with_sales lws
INNER JOIN users ON lws.sellerid = users.userid
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 100

,sellerid,name,unrealized_sales
0,25428,Jaime Wagner,58395.0
1,24896,Macey Ortiz,53086.0
2,49322,Dustin Vincent,50914.0
3,36926,Audrey Barber,50345.0
4,45819,Kelly Barrett,49826.0
...,...,...,...
95,35926,Ulysses Kinney,38895.0
96,45372,Lysandra Sanchez,38862.0
97,48188,Caesar Parrish,38847.0
98,25373,Jakeem Byrd,38847.0


Looks like Jaime Wagner had 58k of unrealized sales!

## Visualize sales over time

In [38]:
-- Show total pricepaid on a weekly basis
SELECT
	DATE_TRUNC('week', saletime) AS sales_week,
	SUM(pricepaid) AS total_sales
FROM public.sales
GROUP BY 1
ORDER BY 1

,sales_week,total_sales
0,2007-12-31 00:00:00+00:00,244991.0
1,2008-01-07 00:00:00+00:00,734374.0
2,2008-01-14 00:00:00+00:00,1206758.0
3,2008-01-21 00:00:00+00:00,1613119.0
4,2008-01-28 00:00:00+00:00,1875270.0
5,2008-02-04 00:00:00+00:00,2013802.0
6,2008-02-11 00:00:00+00:00,2400791.0
7,2008-02-18 00:00:00+00:00,2481096.0
8,2008-02-25 00:00:00+00:00,2080490.0
9,2008-03-03 00:00:00+00:00,2419824.0


In [39]:
import plotly.express as px
px.line(sales_over_time, x='sales_week', y='total_sales')